
# p03 Anchor Analysis

このノートブックは **Dual Momentum** と **Short-term Momentum (high-turnover winner)** を対象に、
- Exit taxonomy（離脱理由）
- Crowding / Factor Decay アンカー
- 可視化（時系列、レジーム、イベント、サバイバル）

を一貫して検証する。

制約:
- `p00/p01` の実装思想（シグナル計算と月次形成→翌月保有）は維持
- `p02` との厳密整合は要求しない
- ファイル出力は行わず、Notebook上で結果を確認する


In [ ]:

from dataclasses import dataclass
from typing import Dict, Tuple, List

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

pd.set_option('display.max_columns', 200)
pd.set_option('display.width', 200)

COST_BPS = 10
TOP_K = 3
CORR_WINDOW = 12

DATA_PATH = 'data/us_sector_etf_9_ohlc.csv'


In [ ]:

# -------------------------
# Data and signal builders
# -------------------------
def rolling_zscore(df: pd.DataFrame, window: int = 12, min_periods: int = 12) -> pd.DataFrame:
    mu = df.rolling(window=window, min_periods=min_periods).mean()
    sd = df.rolling(window=window, min_periods=min_periods).std(ddof=1)
    return (df - mu) / sd


def monthly_signals_from_daily(adj_close_daily: pd.DataFrame, volume_daily: pd.DataFrame):
    monthly_price = adj_close_daily.resample('ME').last()
    monthly_ret = monthly_price.pct_change()
    monthly_vol_sum = volume_daily.resample('ME').sum(min_count=1)

    # Dual momentum signal (same as p00)
    momentum_signal = monthly_price.shift(2) / monthly_price.shift(12) - 1

    # Turnover proxy for ST-MoM (same spirit as p01)
    turnover_base = np.log(monthly_vol_sum.replace(0, np.nan))
    turnover_signal = rolling_zscore(turnover_base, window=12, min_periods=12)

    return monthly_price, monthly_ret, monthly_vol_sum, momentum_signal, turnover_signal


df = pd.read_csv(DATA_PATH, header=[0, 1], index_col=0)
df.index = pd.to_datetime(df.index)

adj_close = df['Adj Close'].astype(float).ffill()
volume = df['Volume'].astype(float)

monthly_price, monthly_ret, monthly_vol, momentum_signal, turnover_signal = monthly_signals_from_daily(adj_close, volume)

print('daily rows:', len(df), 'monthly rows:', len(monthly_ret), 'assets:', list(monthly_ret.columns))
monthly_ret.tail(3)


In [ ]:

# -------------------------
# Strategy builders
# -------------------------
@dataclass
class StrategyResult:
    name: str
    ret_gross: pd.Series
    turnover: pd.Series
    ret_net: pd.Series
    weights: pd.DataFrame


def build_dualmom_strategy(
    monthly_ret: pd.DataFrame,
    momentum_signal: pd.DataFrame,
    strategy_name: str = 'DualMoM_W',
    top_k: int = 3,
    cost_bps: float = 10.0,
) -> StrategyResult:
    months = monthly_ret.index
    assets = list(monthly_ret.columns)

    w_list = []
    gross = []
    one_way_turn = []
    prev_w = pd.Series(0.0, index=assets)

    for i in range(len(months) - 1):
        m = months[i]
        h = months[i + 1]

        r_form = monthly_ret.loc[m]
        r_hold = monthly_ret.loc[h]
        s_form = momentum_signal.loc[m]

        valid = r_form.notna() & r_hold.notna() & s_form.notna()
        if valid.sum() < 4:
            continue

        score = s_form[valid]
        score = score.where(score > 0)
        chosen = score.sort_values(ascending=False).dropna().head(top_k).index
        if len(chosen) == 0:
            continue

        w = pd.Series(0.0, index=assets)
        w.loc[chosen] = 1.0 / len(chosen)

        g = float((w * r_hold).sum())
        t = float(0.5 * (w - prev_w).abs().sum())

        w_list.append(w.rename(h))
        gross.append((h, g))
        one_way_turn.append((h, t))
        prev_w = w

    if not w_list:
        raise ValueError(f'No valid monthly portfolios for {strategy_name}')

    W = pd.DataFrame(w_list).sort_index()
    ret_gross = pd.Series(dict(gross), name=f'{strategy_name}_gross').sort_index()
    turn = pd.Series(dict(one_way_turn), name=f'{strategy_name}_turnover').sort_index()
    ret_net = (ret_gross - turn * (cost_bps / 10000.0)).rename(f'{strategy_name}_net')

    return StrategyResult(strategy_name, ret_gross, turn, ret_net, W)


def build_stmom_strategy(
    monthly_ret: pd.DataFrame,
    turnover_signal: pd.DataFrame,
    strategy_name: str = 'STMoM_WH',
    momentum_high: bool = True,
    turnover_high: bool = True,
    cost_bps: float = 10.0,
) -> StrategyResult:
    months = monthly_ret.index
    assets = list(monthly_ret.columns)

    w_list = []
    gross = []
    one_way_turn = []
    prev_w = pd.Series(0.0, index=assets)

    for i in range(len(months) - 1):
        m = months[i]
        h = months[i + 1]

        r_form = monthly_ret.loc[m]
        r_hold = monthly_ret.loc[h]
        t_form = turnover_signal.loc[m]

        valid = r_form.notna() & r_hold.notna() & t_form.notna()
        if valid.sum() < 4:
            continue

        rv = r_form[valid]
        tv = t_form[valid]

        r_med = float(rv.median())
        t_med = float(tv.median())

        winners = set(rv.index[rv >= r_med])
        losers = set(rv.index[rv < r_med])
        high_turn = set(tv.index[tv >= t_med])
        low_turn = set(tv.index[tv < t_med])

        if turnover_high:
            long_pool = winners.intersection(high_turn)
            short_pool = losers.intersection(high_turn)
        else:
            long_pool = winners.intersection(low_turn)
            short_pool = losers.intersection(low_turn)

        chosen = sorted(long_pool) if momentum_high else sorted(short_pool)
        if len(chosen) == 0:
            continue

        w = pd.Series(0.0, index=assets)
        w.loc[chosen] = 1.0 / len(chosen)

        g = float((w * r_hold).sum())
        t = float(0.5 * (w - prev_w).abs().sum())

        w_list.append(w.rename(h))
        gross.append((h, g))
        one_way_turn.append((h, t))
        prev_w = w

    if not w_list:
        raise ValueError(f'No valid monthly portfolios for {strategy_name}')

    W = pd.DataFrame(w_list).sort_index()
    ret_gross = pd.Series(dict(gross), name=f'{strategy_name}_gross').sort_index()
    turn = pd.Series(dict(one_way_turn), name=f'{strategy_name}_turnover').sort_index()
    ret_net = (ret_gross - turn * (cost_bps / 10000.0)).rename(f'{strategy_name}_net')

    return StrategyResult(strategy_name, ret_gross, turn, ret_net, W)


def annualized_metrics(ret: pd.Series, periods_per_year: int = 12) -> dict:
    ret = ret.dropna()
    if len(ret) == 0:
        return {'n_months': 0, 'CAGR': np.nan, 'Vol': np.nan, 'Sharpe': np.nan, 'MaxDD': np.nan, 'HitRate': np.nan}

    equity = (1 + ret).cumprod()
    years = len(ret) / periods_per_year
    cagr = equity.iloc[-1] ** (1 / years) - 1 if years > 0 else np.nan
    vol = ret.std(ddof=1) * np.sqrt(periods_per_year)
    sharpe = (ret.mean() * periods_per_year) / vol if vol and vol > 0 else np.nan
    dd = equity / equity.cummax() - 1
    maxdd = dd.min()
    hit = (ret > 0).mean()

    return {
        'n_months': int(len(ret)),
        'CAGR': float(cagr),
        'Vol': float(vol),
        'Sharpe': float(sharpe),
        'MaxDD': float(maxdd),
        'HitRate': float(hit),
    }


In [ ]:

# -------------------------
# Exit taxonomy and anchor utils
# -------------------------
def compute_in_port(W: pd.DataFrame, eps: float = 0.0) -> pd.DataFrame:
    return (W.fillna(0.0) > eps)


def compute_exit_events(W: pd.DataFrame) -> pd.DataFrame:
    in_port = compute_in_port(W)
    prev = in_port.shift(1).fillna(False)
    return prev & (~in_port)


def iqr(x: pd.Series) -> float:
    x = x.dropna()
    if len(x) == 0:
        return np.nan
    return float(x.quantile(0.75) - x.quantile(0.25))


def spearman_corr(a: pd.Series, b: pd.Series) -> float:
    df = pd.concat([a, b], axis=1).dropna()
    if len(df) < 3:
        return np.nan
    return float(df.iloc[:, 0].rank().corr(df.iloc[:, 1].rank()))


def top_bottom_spread(score: pd.Series, fwd_ret: pd.Series, q: float = 0.3) -> float:
    df = pd.concat([score.rename('score'), fwd_ret.rename('ret')], axis=1).dropna()
    if len(df) < 6:
        return np.nan
    q_lo = df['score'].quantile(q)
    q_hi = df['score'].quantile(1 - q)
    top = df.loc[df['score'] >= q_hi, 'ret']
    bot = df.loc[df['score'] <= q_lo, 'ret']
    if len(top) == 0 or len(bot) == 0:
        return np.nan
    return float(top.mean() - bot.mean())


def avg_offdiag_corr(R: pd.DataFrame) -> float:
    if R.shape[1] < 2:
        return np.nan
    C = R.corr()
    vals = C.values
    off = vals[~np.eye(vals.shape[0], dtype=bool)]
    if off.size == 0:
        return np.nan
    return float(np.nanmean(off))


def eig1_share(R: pd.DataFrame) -> float:
    if R.shape[1] < 2:
        return np.nan
    X = R.dropna(axis=0, how='any')
    if len(X) < 3:
        return np.nan
    cov = X.cov().values
    try:
        w = np.linalg.eigvalsh(cov)
    except np.linalg.LinAlgError:
        return np.nan
    s = float(np.nansum(w))
    if not np.isfinite(s) or s <= 0:
        return np.nan
    return float(np.nanmax(w) / s)


def dual_formation_mapping(monthly_ret: pd.DataFrame, momentum_signal: pd.DataFrame, top_k: int = 3):
    months = monthly_ret.index
    assets = list(monthly_ret.columns)

    hold_to_form = {}
    score_raw_by_hold = {}
    eligible_by_hold = {}
    selected_by_hold = {}
    valid_by_hold = {}

    for i in range(len(months) - 1):
        m = months[i]
        h = months[i + 1]

        r_form = monthly_ret.loc[m]
        r_hold = monthly_ret.loc[h]
        s_form = momentum_signal.loc[m]

        valid = r_form.notna() & r_hold.notna() & s_form.notna()
        if valid.sum() < 4:
            continue

        score_raw = s_form[valid]
        eligible = score_raw > 0
        score_pos = score_raw.where(eligible)

        selected = pd.Series(False, index=assets)
        if not score_pos.dropna().empty:
            chosen = score_pos.sort_values(ascending=False).dropna().head(top_k).index
            selected.loc[chosen] = True

        sr = pd.Series(np.nan, index=assets)
        sr.loc[score_raw.index] = score_raw.values
        el = pd.Series(False, index=assets)
        el.loc[eligible.index] = eligible.values
        va = pd.Series(False, index=assets)
        va.loc[valid.index] = valid.values

        hold_to_form[h] = m
        score_raw_by_hold[h] = sr
        eligible_by_hold[h] = el
        selected_by_hold[h] = selected
        valid_by_hold[h] = va

    score_raw_h = pd.DataFrame(score_raw_by_hold).T.reindex(columns=assets)
    eligible_h = pd.DataFrame(eligible_by_hold).T.reindex(columns=assets)
    selected_h = pd.DataFrame(selected_by_hold).T.reindex(columns=assets)
    valid_h = pd.DataFrame(valid_by_hold).T.reindex(columns=assets)

    return hold_to_form, score_raw_h, eligible_h, selected_h, valid_h


def st_formation_mapping(monthly_ret: pd.DataFrame, turnover_signal: pd.DataFrame, momentum_high: bool = True, turnover_high: bool = True):
    months = monthly_ret.index
    assets = list(monthly_ret.columns)

    hold_to_form = {}
    valid_by_hold = {}
    score_r_by_hold = {}
    score_t_by_hold = {}
    selected_by_hold = {}
    winner_by_hold = {}
    hturn_by_hold = {}

    for i in range(len(months) - 1):
        m = months[i]
        h = months[i + 1]

        r_form = monthly_ret.loc[m]
        r_hold = monthly_ret.loc[h]
        t_form = turnover_signal.loc[m]

        valid = r_form.notna() & r_hold.notna() & t_form.notna()
        if valid.sum() < 4:
            continue

        rv = r_form[valid]
        tv = t_form[valid]

        r_med = float(rv.median())
        t_med = float(tv.median())

        is_w = rv >= r_med
        is_ht = tv >= t_med

        if momentum_high and turnover_high:
            selected = is_w & is_ht
        elif momentum_high and (not turnover_high):
            selected = is_w & (~is_ht)
        elif (not momentum_high) and turnover_high:
            selected = (~is_w) & is_ht
        else:
            selected = (~is_w) & (~is_ht)

        va = pd.Series(False, index=assets)
        va.loc[valid.index] = valid.values
        sr = pd.Series(np.nan, index=assets)
        sr.loc[rv.index] = rv.values
        st = pd.Series(np.nan, index=assets)
        st.loc[tv.index] = tv.values
        se = pd.Series(False, index=assets)
        se.loc[selected.index] = selected.values
        iw = pd.Series(False, index=assets)
        iw.loc[is_w.index] = is_w.values
        ih = pd.Series(False, index=assets)
        ih.loc[is_ht.index] = is_ht.values

        hold_to_form[h] = m
        valid_by_hold[h] = va
        score_r_by_hold[h] = sr
        score_t_by_hold[h] = st
        selected_by_hold[h] = se
        winner_by_hold[h] = iw
        hturn_by_hold[h] = ih

    valid_h = pd.DataFrame(valid_by_hold).T.reindex(columns=assets)
    score_r_h = pd.DataFrame(score_r_by_hold).T.reindex(columns=assets)
    score_t_h = pd.DataFrame(score_t_by_hold).T.reindex(columns=assets)
    selected_h = pd.DataFrame(selected_by_hold).T.reindex(columns=assets)
    winner_h = pd.DataFrame(winner_by_hold).T.reindex(columns=assets)
    hturn_h = pd.DataFrame(hturn_by_hold).T.reindex(columns=assets)

    return hold_to_form, valid_h, score_r_h, score_t_h, selected_h, winner_h, hturn_h


def dual_exit_reasons(W: pd.DataFrame, monthly_ret: pd.DataFrame, momentum_signal: pd.DataFrame, top_k: int = 3):
    exit_df = compute_exit_events(W)
    assets = list(W.columns)

    hold_to_form, score_raw_h, eligible_h, selected_h, valid_h = dual_formation_mapping(monthly_ret, momentum_signal, top_k=top_k)

    reasons = pd.DataFrame('', index=W.index, columns=assets)
    for h in W.index:
        for a in assets:
            if not bool(exit_df.loc[h, a]):
                continue
            if (h not in hold_to_form) or (not bool(valid_h.loc[h, a])):
                reasons.loc[h, a] = 'DATA_OUT'
            elif not bool(eligible_h.loc[h, a]):
                reasons.loc[h, a] = 'ABS_FAIL'
            elif not bool(selected_h.loc[h, a]):
                reasons.loc[h, a] = 'RANK_OUT'
            else:
                reasons.loc[h, a] = 'CONSTRAINT_OR_TIE'

    return exit_df, reasons, hold_to_form, score_raw_h, selected_h


def st_exit_reasons(
    W: pd.DataFrame,
    monthly_ret: pd.DataFrame,
    turnover_signal: pd.DataFrame,
    momentum_high: bool = True,
    turnover_high: bool = True,
):
    exit_df = compute_exit_events(W)
    assets = list(W.columns)

    hold_to_form, valid_h, score_r_h, score_t_h, selected_h, winner_h, hturn_h = st_formation_mapping(
        monthly_ret,
        turnover_signal,
        momentum_high=momentum_high,
        turnover_high=turnover_high,
    )

    reasons = pd.DataFrame('', index=W.index, columns=assets)
    prev_h = None
    for h in W.index:
        for a in assets:
            if not bool(exit_df.loc[h, a]):
                continue

            if (h not in hold_to_form) or (not bool(valid_h.loc[h, a])):
                reasons.loc[h, a] = 'DATA_OUT'
                continue

            if (prev_h is None) or (prev_h not in hold_to_form) or (not bool(valid_h.loc[prev_h, a])):
                reasons.loc[h, a] = 'BUCKET_CHANGE'
                continue

            flip_mom = bool(winner_h.loc[h, a]) != bool(winner_h.loc[prev_h, a])
            flip_turn = bool(hturn_h.loc[h, a]) != bool(hturn_h.loc[prev_h, a])

            if flip_mom and flip_turn:
                reasons.loc[h, a] = 'BOTH_FLIP'
            elif flip_mom:
                reasons.loc[h, a] = 'MOM_BUCKET_FLIP'
            elif flip_turn:
                reasons.loc[h, a] = 'TURN_BUCKET_FLIP'
            elif not bool(selected_h.loc[h, a]):
                reasons.loc[h, a] = 'BUCKET_CHANGE'
            else:
                reasons.loc[h, a] = 'CONSTRAINT_OR_TIE'
        prev_h = h

    return exit_df, reasons, hold_to_form, score_r_h, score_t_h, selected_h


def compute_anchors(
    W: pd.DataFrame,
    monthly_ret: pd.DataFrame,
    hold_to_form: Dict[pd.Timestamp, pd.Timestamp],
    score_h: pd.DataFrame,
    selected_h: pd.DataFrame,
    corr_window: int = 12,
) -> pd.DataFrame:
    rows = []
    for h in W.index:
        if h not in hold_to_form:
            continue
        m = hold_to_form[h]
        if m not in monthly_ret.index:
            continue

        mi = monthly_ret.index.get_loc(m)
        lo = max(0, mi - corr_window + 1)
        win = monthly_ret.iloc[lo:mi + 1, :]

        if h in selected_h.index:
            sel_assets = list(selected_h.columns[selected_h.loc[h].fillna(False).values])
        else:
            sel_assets = list(W.columns[(W.loc[h].fillna(0.0) > 0).values])

        R = win[sel_assets].dropna(axis=1, how='all') if len(sel_assets) > 0 else pd.DataFrame(index=win.index)
        crowd_corr = avg_offdiag_corr(R) if len(sel_assets) >= 2 else np.nan
        crowd_eig = eig1_share(R) if len(sel_assets) >= 2 else np.nan

        score = score_h.loc[h] if h in score_h.index else pd.Series(index=W.columns, dtype=float)
        fwd = monthly_ret.loc[h] if h in monthly_ret.index else pd.Series(index=W.columns, dtype=float)

        rank_ic = spearman_corr(score, fwd)
        spread = top_bottom_spread(score, fwd, q=0.3)
        disp = iqr(score)

        rows.append({
            'hold_month': h,
            'form_month': m,
            'n_sel': len(sel_assets),
            'crowd_avg_corr': crowd_corr,
            'crowd_eig1_share': crowd_eig,
            'rank_ic': rank_ic,
            'spread_top_bottom': spread,
            'dispersion_iqr': disp,
        })

    if not rows:
        return pd.DataFrame(columns=['form_month', 'n_sel', 'crowd_avg_corr', 'crowd_eig1_share', 'rank_ic', 'spread_top_bottom', 'dispersion_iqr'])

    return pd.DataFrame(rows).set_index('hold_month').sort_index()


def make_summary(W: pd.DataFrame, exit_df: pd.DataFrame, res: StrategyResult) -> pd.DataFrame:
    in_port = compute_in_port(W)
    holdings_prev = in_port.shift(1).sum(axis=1).astype(float)
    exit_count = exit_df.sum(axis=1).astype(float)

    exit_rate = pd.Series(np.nan, index=W.index, dtype=float)
    mask = holdings_prev > 0
    exit_rate.loc[mask] = (exit_count.loc[mask] / holdings_prev.loc[mask]).astype(float)

    return pd.DataFrame({
        'holdings_prev': holdings_prev,
        'exit_count': exit_count,
        'exit_rate': exit_rate,
        'turnover': res.turnover.reindex(W.index),
        'ret_gross': res.ret_gross.reindex(W.index),
        'ret_net': res.ret_net.reindex(W.index),
    })


def holding_durations(W: pd.DataFrame) -> List[int]:
    in_port = compute_in_port(W)
    durations = []
    for col in in_port.columns:
        run = 0
        for val in in_port[col].fillna(False).values:
            if val:
                run += 1
            else:
                if run > 0:
                    durations.append(run)
                run = 0
        if run > 0:
            durations.append(run)
    return durations


def evaluate_hypothesis(summary: pd.DataFrame, anchors: pd.DataFrame) -> pd.Series:
    df = summary.join(anchors[['crowd_eig1_share', 'rank_ic', 'dispersion_iqr']], how='left')

    q_hi = df['crowd_eig1_share'].quantile(0.75)
    q_lo = df['crowd_eig1_share'].quantile(0.25)

    high = df[df['crowd_eig1_share'] >= q_hi]
    low = df[df['crowd_eig1_share'] <= q_lo]

    corr_ce_er = df[['crowd_eig1_share', 'exit_rate']].corr().iloc[0, 1]
    corr_ic_er = df[['rank_ic', 'exit_rate']].corr().iloc[0, 1]

    return pd.Series({
        'n_obs': len(df),
        'corr_crowd_exit_rate': corr_ce_er,
        'corr_rankic_exit_rate': corr_ic_er,
        'exit_rate_high_crowd': high['exit_rate'].mean(),
        'exit_rate_low_crowd': low['exit_rate'].mean(),
        'rank_ic_mean': df['rank_ic'].mean(),
        'dispersion_mean': df['dispersion_iqr'].mean(),
    })


In [ ]:

# -------------------------
# Run strategies and compute facts
# -------------------------
res_dual = build_dualmom_strategy(
    monthly_ret=monthly_ret,
    momentum_signal=momentum_signal,
    strategy_name='DualMoM_W',
    top_k=TOP_K,
    cost_bps=COST_BPS,
)

res_st = build_stmom_strategy(
    monthly_ret=monthly_ret,
    turnover_signal=turnover_signal,
    strategy_name='STMoM_WH',
    momentum_high=True,
    turnover_high=True,
    cost_bps=COST_BPS,
)

W_dual = res_dual.weights.copy()
W_st = res_st.weights.copy()

exit_dual, reason_dual, hold_to_form_dual, score_dual_h, selected_dual_h = dual_exit_reasons(
    W_dual, monthly_ret, momentum_signal, top_k=TOP_K
)
anchors_dual = compute_anchors(W_dual, monthly_ret, hold_to_form_dual, score_dual_h, selected_dual_h, corr_window=CORR_WINDOW)
summary_dual = make_summary(W_dual, exit_dual, res_dual)

exit_st, reason_st, hold_to_form_st, score_stret_h, score_stturn_h, selected_st_h = st_exit_reasons(
    W_st,
    monthly_ret,
    turnover_signal,
    momentum_high=True,
    turnover_high=True,
)
anchors_st = compute_anchors(W_st, monthly_ret, hold_to_form_st, score_stret_h, selected_st_h, corr_window=CORR_WINDOW)
summary_st = make_summary(W_st, exit_st, res_st)

perf = pd.DataFrame([
    {'strategy': 'DualMoM_W', **annualized_metrics(res_dual.ret_net)},
    {'strategy': 'STMoM_WH', **annualized_metrics(res_st.ret_net)},
]).set_index('strategy')

dual_hyp = evaluate_hypothesis(summary_dual, anchors_dual).rename('DualMoM_W')
st_hyp = evaluate_hypothesis(summary_st, anchors_st).rename('STMoM_WH')
hyp_df = pd.concat([dual_hyp, st_hyp], axis=1).T

reason_count_dual = reason_dual.replace('', np.nan).stack().value_counts().rename('count')
reason_count_st = reason_st.replace('', np.nan).stack().value_counts().rename('count')

print('Performance summary:')
display(perf)
print('Hypothesis diagnostics:')
display(hyp_df)
print('Exit reason counts (Dual):')
display(reason_count_dual)
print('Exit reason counts (ST):')
display(reason_count_st)


In [ ]:

# -------------------------
# Plot helpers
# -------------------------
def plot_holdings_matrix(W: pd.DataFrame, exit_df: pd.DataFrame, reasons: pd.DataFrame, title: str):
    W0 = W.fillna(0.0)
    fig, ax = plt.subplots(figsize=(12, 5))
    im = ax.imshow(W0.values, aspect='auto', interpolation='nearest')
    ax.set_title(title)
    ax.set_yticks(range(len(W0.index)))
    ax.set_yticklabels([d.strftime('%Y-%m') for d in W0.index], fontsize=8)
    ax.set_xticks(range(len(W0.columns)))
    ax.set_xticklabels(W0.columns, rotation=90, fontsize=8)

    marker_map = {
        'ABS_FAIL': 'x',
        'RANK_OUT': 'o',
        'DATA_OUT': 's',
        'MOM_BUCKET_FLIP': 'x',
        'TURN_BUCKET_FLIP': 'o',
        'BOTH_FLIP': 's',
        'BUCKET_CHANGE': 'd',
        'CONSTRAINT_OR_TIE': '*',
    }

    xs, ys, ms = [], [], []
    for yi, h in enumerate(W0.index):
        for xi, a in enumerate(W0.columns):
            if bool(exit_df.loc[h, a]):
                r = reasons.loc[h, a]
                m = marker_map.get(r, '.')
                xs.append(xi)
                ys.append(yi)
                ms.append(m)

    for m in sorted(set(ms)):
        pts = [(x, y) for x, y, mm in zip(xs, ys, ms) if mm == m]
        if not pts:
            continue
        px = [p[0] for p in pts]
        py = [p[1] for p in pts]
        ax.scatter(px, py, marker=m, s=28, linewidths=1)

    fig.colorbar(im, ax=ax, fraction=0.02, pad=0.02)
    fig.tight_layout()
    plt.show()


def plot_exit_reason_timeseries(reason_df: pd.DataFrame, summary_df: pd.DataFrame, title: str):
    reasons = sorted([r for r in pd.unique(reason_df.values.ravel()) if isinstance(r, str) and r != ''])
    counts = {r: (reason_df == r).sum(axis=1) for r in reasons}

    idx = summary_df.index
    x = np.arange(len(idx))

    fig, ax = plt.subplots(figsize=(12, 4))
    bottom = np.zeros(len(idx))
    for r in reasons:
        y = counts[r].reindex(idx).fillna(0).values
        ax.bar(x, y, bottom=bottom, label=r)
        bottom += y

    ax2 = ax.twinx()
    ax2.plot(x, summary_df['turnover'].reindex(idx).values, linewidth=1.5)

    ax.set_title(title)
    ax.set_ylabel('Exit count')
    ax2.set_ylabel('Turnover (one-way)')
    ax.set_xticks(x)
    ax.set_xticklabels([d.strftime('%Y-%m') for d in idx], rotation=90, fontsize=8)
    ax.legend(loc='upper left', fontsize=8, ncol=2)
    fig.tight_layout()
    plt.show()


def plot_regime_map(anchors: pd.DataFrame, summary: pd.DataFrame, title: str):
    df = anchors.join(summary[['exit_rate']], how='left').dropna(subset=['dispersion_iqr', 'crowd_eig1_share'])

    x = df['dispersion_iqr'].values
    y = df['crowd_eig1_share'].values
    c = df['rank_ic'].values
    s = (df['exit_rate'].fillna(0).values * 800) + 20

    fig, ax = plt.subplots(figsize=(6, 5))
    sc = ax.scatter(x, y, c=c, s=s)
    ax.set_title(title)
    ax.set_xlabel('dispersion_iqr')
    ax.set_ylabel('crowd_eig1_share')
    fig.colorbar(sc, ax=ax, fraction=0.04, pad=0.02, label='rank_ic')
    fig.tight_layout()
    plt.show()


def plot_event_study(summary: pd.DataFrame, anchors: pd.DataFrame, event_months: pd.DatetimeIndex, cols: List[str], window: int, title: str):
    idx = summary.index
    pos = {d: i for i, d in enumerate(idx)}

    rows = []
    for h in event_months:
        if h not in pos:
            continue
        i0 = pos[h]
        lo = max(0, i0 - window)
        hi = min(len(idx) - 1, i0 + window)
        rel = np.arange(lo, hi + 1) - i0

        block = pd.DataFrame(index=rel)
        for col in cols:
            if col in summary.columns:
                s = summary[col]
            else:
                s = anchors[col]
            block[col] = s.reindex(idx[lo:hi + 1]).values
        rows.append(block)

    if len(rows) == 0:
        print('No events for', title)
        return

    panel = pd.concat(rows, keys=range(len(rows)), names=['event', 'rel'])
    mean_path = panel.groupby(level='rel').mean(numeric_only=True)

    fig, axes = plt.subplots(len(cols), 1, figsize=(10, 2.3 * len(cols)), sharex=True)
    if len(cols) == 1:
        axes = [axes]

    for ax, col in zip(axes, cols):
        ax.plot(mean_path.index.values, mean_path[col].values)
        ax.axvline(0, linestyle='--', linewidth=1)
        ax.set_ylabel(col)
        ax.grid(alpha=0.3)

    axes[0].set_title(title)
    axes[-1].set_xlabel('months relative to event')
    fig.tight_layout()
    plt.show()


def plot_survival(W: pd.DataFrame, title: str):
    durs = holding_durations(W)
    if len(durs) == 0:
        print('No durations:', title)
        return

    max_t = max(durs)
    t_grid = np.arange(1, max_t + 1)
    surv = [np.mean(np.array(durs) >= t) for t in t_grid]

    fig, ax = plt.subplots(figsize=(6, 4))
    ax.step(t_grid, surv, where='post')
    ax.set_title(title)
    ax.set_xlabel('Holding months')
    ax.set_ylabel('Survival prob')
    ax.set_ylim(0, 1.02)
    ax.grid(alpha=0.3)
    fig.tight_layout()
    plt.show()


In [ ]:

# -------------------------
# Visual diagnostics
# -------------------------
plot_holdings_matrix(W_dual, exit_dual, reason_dual, 'DualMoM: Holdings + Exit markers')
plot_exit_reason_timeseries(reason_dual, summary_dual, 'DualMoM: Exit reasons + Turnover')
plot_regime_map(anchors_dual, summary_dual, 'DualMoM: Regime map')

q_dual = anchors_dual['crowd_eig1_share'].quantile(0.75)
events_dual = anchors_dual.index[anchors_dual['crowd_eig1_share'] >= q_dual]
plot_event_study(summary_dual, anchors_dual, events_dual, ['exit_rate', 'ret_net', 'crowd_eig1_share', 'rank_ic'], window=12, title='DualMoM: Event study (high crowding)')
plot_survival(W_dual, 'DualMoM: Holding survival')

plot_holdings_matrix(W_st, exit_st, reason_st, 'STMoM: Holdings + Exit markers')
plot_exit_reason_timeseries(reason_st, summary_st, 'STMoM: Exit reasons + Turnover')
plot_regime_map(anchors_st, summary_st, 'STMoM: Regime map')

q_st = anchors_st['crowd_eig1_share'].quantile(0.75)
events_st = anchors_st.index[anchors_st['crowd_eig1_share'] >= q_st]
plot_event_study(summary_st, anchors_st, events_st, ['exit_rate', 'ret_net', 'crowd_eig1_share', 'rank_ic'], window=12, title='STMoM: Event study (high crowding)')
plot_survival(W_st, 'STMoM: Holding survival')


In [ ]:

# -------------------------
# Compact report table for review_phaseD.md
# -------------------------
compact = pd.DataFrame({
    'dual': {
        'n_months': int(len(summary_dual)),
        'mean_exit_rate': float(summary_dual['exit_rate'].mean()),
        'mean_turnover': float(summary_dual['turnover'].mean()),
        'mean_rank_ic': float(anchors_dual['rank_ic'].mean()),
        'mean_crowd_eig1_share': float(anchors_dual['crowd_eig1_share'].mean()),
    },
    'st': {
        'n_months': int(len(summary_st)),
        'mean_exit_rate': float(summary_st['exit_rate'].mean()),
        'mean_turnover': float(summary_st['turnover'].mean()),
        'mean_rank_ic': float(anchors_st['rank_ic'].mean()),
        'mean_crowd_eig1_share': float(anchors_st['crowd_eig1_share'].mean()),
    },
}).T

print('compact summary:')
display(compact)

print('Phase D rerun command:')
print('/Users/kencharoff/workspace/envs/base/.venv/bin/python -m jupyter nbconvert --to notebook --execute notebooks/p03_anchor_analysis.ipynb --output /tmp/p03_anchor_analysis.executed.ipynb')
